In [1]:
import pandas as pd
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from deepchembed.dimreducer import DeepAutoEncoder

Using TensorFlow backend.


ModuleNotFoundError: No module named 'deepchembed'

In [2]:
data = pd.read_csv('BOD_biclass_139rdkit.csv')

In [3]:
data.head()

,SMILES,bod_class,exactmw,lipinskiHBA,lipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumHeteroatoms,NumAmideBonds,...,MQN33,MQN34,MQN35,MQN36,MQN37,MQN38,MQN39,MQN40,MQN41,MQN42
0,CCC1(C(=O)NC(=O)NC1=O)c2ccccc2,0,232.084792,5.0,2.0,2.0,2.0,3.0,5.0,4.0,...,0,0,0,2,0,0,0,0,0,0
1,CC(O)C(O)=O,1,90.031694,3.0,2.0,1.0,2.0,2.0,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,0,351.914689,0.0,0.0,2.0,0.0,0.0,5.0,0.0,...,0,0,0,2,0,0,0,0,0,0
3,OC(=O)c1c(Cl)ccc(Cl)c1Cl,1,223.919862,2.0,1.0,1.0,1.0,1.0,5.0,0.0,...,0,0,0,1,0,0,0,0,0,0
4,OCC(O)C(O)C(O)C(O)CO,1,182.079038,6.0,6.0,5.0,6.0,6.0,6.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.shape

(1063, 141)

In [5]:
data_139 = data.iloc[:,2:141]
data_139.shape

(1063, 139)

In [6]:
data_train = data_139.as_matrix()
data_train.shape

/Users/hunting/miniconda3/envs/deepchembed/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(1063, 139)

In [7]:
def autoencoder(dims, final_nodes, act='relu'):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: a list contains the node numbers in each layer of encoder. The decoder is symmetric with encoder. 
        So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation function
        final_nodes: the final target dimension for dimension reduction, which interconnects the encoder and decoder
    return:
        Model of autoencoder
    """
    n_stacks = len(dims) - 1
    # input
    x = Input(shape=(dims[0],), name='input')
    h = x

    # internal layers in encoder
    for i in range(n_stacks-1):
        h = Dense(dims[i + 1], activation=act, name='encoder_%d' % i)(h)

    # inter-connected layer between encoder and decoder
    h = Dense(final_nodes, name='encoder_%d' % (n_stacks - 1))(h)  #features are extracted from here

    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        h = Dense(dims[i], activation=act, name='decoder_%d' % i)(h)

    # output
    h = Dense(dims[0], name='decoder_0')(h)

    return Model(inputs=x, outputs=h)

Build and compile a 4 layers autoencoder model, the final_nodes are set to 5 here

In [8]:
dim = np.linspace(139,5,4,dtype = int)
dim

array([139,  94,  49,   5])

In [9]:
final_nodes = 5

In [10]:
autoencoder = autoencoder (dim, final_nodes)

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 139)               0         
_________________________________________________________________
encoder_0 (Dense)            (None, 94)                13160     
_________________________________________________________________
encoder_1 (Dense)            (None, 49)                4655      
_________________________________________________________________
encoder_2 (Dense)            (None, 5)                 250       
_________________________________________________________________
decoder_2 (Dense)            (None, 49)                294       
_________________________________________________________________
decoder_1 (Dense)            (None, 94)                4700      
_________________________________________________________________
decoder_0 (Dense)            (None, 139)               13205     
Total para

In [11]:
autoencoder.compile(optimizer = 'adadelta', loss = 'mse')

In [12]:
history = autoencoder.fit(data_train,data_train,epochs= 50,batch_size=256,shuffle=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
1063/1063 [==============================] - 1s 1ms/step - loss: 1360.5217
Epoch 2/50
1063/1063 [==============================] - 0s 46us/step - loss: 1269.1243
Epoch 3/50
1063/1063 [==============================] - 0s 40us/step - loss: 892.7531
Epoch 4/50
1063/1063 [==============================] - 0s 38us/step - loss: 805.8568
Epoch 5/50
1063/1063 [==============================] - 0s 217us/step - loss: 746.5337
Epoch 6/50
1063/1063 [==============================] - 0s 39us/step - loss: 722.3785
Epoch 7/50
1063/1063 [==============================] - 0s 40us/step - loss: 685.2290
Epoch 8/50
1063/1063 [==============================] - 0s 39us/step - loss: 681.4400
Epoch 9/50
1063/1063 [==============================] - 0s 38us/step - loss: 651.2661
Epoch 10/50
1063/1063 [==============================] - 0s 39us/step - loss: 601.7112
Epoch 11/50
1063/1063 [==============================] - 0s 38us/step - loss: 560.0702
Ep

In [13]:
print(history.history.keys())

dict_keys(['loss'])


In [14]:
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
a = history.history['loss']

In [ ]:
plt.plot(a)

In [18]:
a

[1360.5216915385406,
 1269.1242911768543,
 892.7530682367305,
 805.8568293373202,
 746.5336573072345,
 722.3785232228147,
 685.2290478308921,
 681.440013996745,
 651.2660886489997,
 601.7112410537275,
 560.0701554048095,
 531.750545828273,
 380.9367232515581,
 549.4730548984244,
 295.12832644472456,
 244.1142509331259,
 206.54968050708214,
 169.13964083681438,
 154.80102185225107,
 133.80875630706186,
 136.5584604114378,
 118.23301176428234,
 144.71424510449816,
 167.6690393342012,
 175.41945421034058,
 140.6141661305755,
 120.68052648937265,
 608.4594336695416,
 84.22427475463637,
 67.35880976808822,
 62.69709347557147,
 54.889915710334385,
 58.1980482408267,
 50.96748912951235,
 50.5972182865089,
 51.13825999894237,
 52.49832096261341,
 52.44431558646689,
 49.21448031306154,
 51.06827160825397,
 48.20155778025415,
 48.08615706579257,
 48.07817523342646,
 47.712268728864494,
 48.09234448322572,
 49.69750049443931,
 42.18159176960153,
 43.52134661526469,
 46.576512922572284,
 44.691986